# Project: Extreme Absenteeism Predictive Analysis

## Table of Content
* [Load and Clean Data](#load-and-clean-data) 
* [Explotory Data Analysis](#Exploratory Data Analysis)
* [Feature Engineering](#Feature Engineering)
* [Model Building](../data/absenteeism_module.py)

<a id='Load and Clean Data'></a>
## Load and Clean Data


In [2]:
# importing packages for this project
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# loading the file
raw_csv_data = pd.read_csv('../data/Absenteeism_data.csv')
raw_csv_data

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
695,17,10,23/05/2018,179,22,40,237.656,22,2,2,0,8
696,28,6,23/05/2018,225,26,28,237.656,24,1,1,2,3
697,18,10,24/05/2018,330,16,28,237.656,25,2,0,0,8
698,25,23,24/05/2018,235,16,32,237.656,25,3,0,0,2


In [4]:
""" To display all data rowsand columns
pd.options.display.max_columns = None
pd.options.display.max_rows = None """

' To display all data rowsand columns\npd.options.display.max_columns = None\npd.options.display.max_rows = None '

In [5]:
raw_csv_data.describe().corr

<bound method DataFrame.corr of                ID  Reason for Absence  Transportation Expense  \
count  700.000000          700.000000              700.000000   
mean    17.951429           19.411429              222.347143   
std     11.028144            8.356292               66.312960   
min      1.000000            0.000000              118.000000   
25%      9.000000           13.000000              179.000000   
50%     18.000000           23.000000              225.000000   
75%     28.000000           27.000000              260.000000   
max     36.000000           28.000000              388.000000   

       Distance to Work         Age  Daily Work Load Average  Body Mass Index  \
count        700.000000  700.000000               700.000000       700.000000   
mean          29.892857   36.417143               271.801774        26.737143   
std           14.804446    6.379083                40.021804         4.254701   
min            5.000000   27.000000               205.9170

In [ ]:
raw_csv_data.info()

In [ ]:
raw_csv_data.isnull().sum()

Let's create a copy of the dataframe before processing

In [ ]:
df = raw_csv_data.copy()

## Removing irrelevant columns

In [ ]:
df = df.drop(["ID"], axis = 1)

In [ ]:
#checking the minimum and maximum values of Reason for absence 
print(df['Reason for Absence'].max())
print(df['Reason for Absence'].min())

In [ ]:
#checking the number of values in 'Reason for Absence' column
len(df['Reason for Absence'].unique()) 

In [ ]:
sorted(df['Reason for Absence'].unique())

These 28 reasons for absence represents an actual reason for absence. as shown in the supporting document. there we can see why 20 isn't amongst the reasons. We need to add numerical meaning to these categorical nominal values by creating dummy variables.
A dummy variable is an explanatory binary variable that equals:
1 if a certain categorical efect is present, and equals 0 if that same effect is absent.

In [ ]:
# using .get_dummies to create dummy variable and to avoid multicollinearity issues we drop the first column which is 'zero reason' variable.
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first = True)
reason_columns

In [ ]:
#checking for rows with missing values
reason_columns['check'] = reason_columns.sum(axis=1)
reason_columns
# expect that each row has a check column value of 1, as an individual can't be absent for 2 reasons.

In [ ]:
# programmatically checking the check values 
reason_columns['check'].unique()

In [ ]:
# dropping the check column as it's confirmed our dummies
reason_columns = reason_columns.drop(['check'], axis = 1)

In [ ]:
reason_columns.columns.values
reason_columns

In [ ]:
# grouping the reason for absence data
reason_type_1 = reason_columns.iloc[:, :14].max(axis=1)
reason_type_2 = reason_columns.iloc[:, 14:17].max(axis=1)
reason_type_3 = reason_columns.iloc[:, 17:20].max(axis=1)
reason_type_4 = reason_columns.iloc[:, 20:].max(axis=1)

### Concatenate Column Values

In [ ]:
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)
df

In [ ]:
df.columns.values

In [ ]:
# renaming the last concatenated columns
column_names = ['Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']

In [ ]:
# adding the new column names to the original dataframe
df.columns = column_names
df.head()

In [ ]:
# drop original reason column
df = df.drop(['Reason for Absence'], axis = 1)

### Reorder Columns

In [ ]:
column_names_rendered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours']

df = df[column_names_rendered]
df.head()

### Checkpoint

In [ ]:
df_reason_mod = df.copy()

### 'Date':

In [ ]:
df_reason_mod['Date']

In [ ]:
type(df_reason_mod['Date'])

In [ ]:
# using timestamp to convert the datetime
df_reason_mod['Date'] = df_reason_mod['Date'].apply(pd.to_datetime, errors='coerce')
df_reason_mod.dtypes

In [ ]:
df_reason_mod['Date']

In [ ]:
df_reason_mod.info()

In [ ]:
# Extracting the 'Year','Month' and 'day' from Date Column.
df_reason_mod['Month'] = df_reason_mod['Date'].dt.month_name()
df_reason_mod['Day'] = df_reason_mod['Date'].dt.day_name()

In [ ]:
# dropping the 'Date' Column
df_reason_mod = df_reason_mod.drop(['Date'], axis = 1)

In [ ]:
df_reason_mod.columns.values

In [ ]:
date_rearranged = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month',
       'Day','Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [ ]:
df_reason_mod = df_reason_mod[date_rearranged]
df_reason_mod

### Checkpoint

In [ ]:
df_reason_date_mod = df_reason_mod.copy()

In [ ]:
df_reason_date_mod.head(9)

Assuming a commonality exists between Education, Children and pets as they represents categorical data, containing integers. Education has no numeric meaning as such will be converted to a dummy variable

In [ ]:
# checking the variables in Education
df_reason_date_mod['Education'].unique()

1 = High School
2 = Graduate
3 = Postgraduate
4 = Master or doctor

In [ ]:
df_reason_date_mod['Education'].value_counts()

Shows that majority of staff has just High School certificate. There group 2-4 will be merged.

In [ ]:
# using the 'map' method to reassign the values in the Education column
df_reason_date_mod['Education'] = df_reason_date_mod['Education'].map({1:0, 2:1, 3:1, 4:1})

In [ ]:
df_reason_date_mod['Education'].unique()

In [ ]:
df_reason_date_mod['Education'].value_counts()

### Checkpoint

In [ ]:
df_preprocessed = df_reason_date_mod.copy()
df_preprocessed.head()

In [ ]:
#df_preprocessed.to_csv('../data/preprocessed.csv', index=False)